In [3]:
# ==============================================================================
# Mini Projet - Analyse de données du Burkina Faso
# ==============================================================================
# Ce script Python réalise toutes les étapes de l'exercice :
# 1. Prétraitement et filtrage d'un fichier Geonames.org pour le Burkina Faso.
# 2. Analyse et extraction de sous-ensembles de données.
# 3. Consolidation des résultats dans un fichier Excel.
#
# Assurez-vous d'avoir téléchargé et décompressé le fichier BF.zip de Geonames.
# Le fichier BF.txt doit se trouver dans le même répertoire que ce script.
#
# ==============================================================================

# ------------------------------------------------------------------------------
# Étape 1 : Configuration et importation des bibliothèques
# ------------------------------------------------------------------------------
# La bibliothèque 'pandas' est essentielle pour la manipulation de données.
import pandas as pd
import os
import openpyxl
# Pour gérer les fichiers Excel, pandas a besoin de la bibliothèque 'openpyxl'.
# Si vous exécutez ce code dans un environnement comme Jupyter ou Google Colab,
# vous pouvez installer les paquets manquants avec la commande suivante.


print("Bibliothèques importées avec succès.")
print("-" * 50)


# ------------------------------------------------------------------------------
# Étape 2 : Prétraitement et filtrage des données brutes
# ------------------------------------------------------------------------------
# Nous lisons le fichier BF.txt. Le code ISO du Burkina Faso est 'BF'.
# Le fichier est délimité par des tabulations et n'a pas d'en-tête,
# nous spécifions donc 'sep='\t'' et 'header=None'.

# CORRECTION DU CHEMIN DU FICHIER :
# Après décompression, BF.txt se trouve souvent dans un sous-dossier 'BF'.
file_path = os.path.join('BF', 'BF.txt')
print(f"Lecture du fichier {file_path}...")

try:
    df_raw = pd.read_csv(file_path, sep='\t', header=None, encoding='utf-8')
except FileNotFoundError:
    print(f"Erreur : Le fichier '{file_path}' n'a pas été trouvé.")
    print("Veuillez vous assurer que vous avez bien décompressé le fichier BF.zip")
    print("et que le fichier BF.txt se trouve dans un sous-dossier nommé 'BF'.")
    raise # Relève l'erreur pour arrêter l'exécution si le fichier n'est pas trouvé.


# Le README de Geonames indique que les colonnes qui nous intéressent sont :
# - geonameid (colonne 0)
# - name (colonne 1)
# - latitude (colonne 4)
# - longitude (colonne 5)
df_filtered = df_raw[[0, 1, 4, 5]].copy()

# Renommage des colonnes pour les rendre plus lisibles
df_filtered.columns = ['ID', 'location_name', 'lat', 'long']

# Sauvegarde du DataFrame filtré dans un nouveau fichier CSV
df_filtered.to_csv('burkina_location.csv', index=False, encoding='utf-8')

print("Fichier 'burkina_location.csv' créé avec succès.")
print("\nAperçu des 5 premières lignes du nouveau fichier :")
print(df_filtered.head())
print("-" * 50)


# ------------------------------------------------------------------------------
# Étape 3 : Opérations sur le fichier CSV
# ------------------------------------------------------------------------------
# Rechargement du fichier CSV pour les analyses suivantes
df_burkina = pd.read_csv('burkina_location.csv')

# 3.1 Extraire les données contenant le nom 'gounghin'
# La méthode str.contains() est utilisée pour rechercher la sous-chaîne.
df_gounghin = df_burkina[df_burkina['location_name'].str.contains('gounghin', case=False, na=False)]
df_gounghin.to_csv('gounghin.csv', index=False)
print("Fichier 'gounghin.csv' créé avec les données contenant 'gounghin'.")
print("\nDonnées pour 'gounghin' :")
print(df_gounghin)
print("-" * 50)


# 3.2 Extraire les lieux dont les noms commencent par une lettre de 'A' à 'P'
# Nous utilisons .str.upper() pour rendre la comparaison insensible à la casse.
df_a_to_p = df_burkina[df_burkina['location_name'].str.upper().str[0].between('A', 'P')]
print("Aperçu des lieux dont le nom commence par une lettre de 'A' à 'P' :")
print(df_a_to_p.head())
print("-" * 50)


# 3.3 Identifier les coordonnées minimales et les noms de lieux correspondants
min_lat = df_a_to_p['lat'].min()
min_long = df_a_to_p['long'].min()

# Trouver le lieu correspondant à la latitude minimale
lieu_min_lat = df_a_to_p[df_a_to_p['lat'] == min_lat]['location_name'].values
# Trouver le lieu correspondant à la longitude minimale
lieu_min_long = df_a_to_p[df_a_to_p['long'] == min_long]['location_name'].values

print(f"Latitude minimale : {min_lat} (Lieu : {lieu_min_lat})")
print(f"Longitude minimale : {min_long} (Lieu : {lieu_min_long})")
print("-" * 50)


# 3.4 Quels sont les lieux dont les coordonnées sont (lat >= 11 et long <= 0.5)
df_specific_coords = df_burkina[(df_burkina['lat'] >= 11) & (df_burkina['long'] <= 0.5)]
print("Lieux dont les coordonnées sont (lat >= 11 et long <= 0.5) :")
print(df_specific_coords)
print("-" * 50)


# ------------------------------------------------------------------------------
# Étape 4 : Création du fichier Excel final
# ------------------------------------------------------------------------------
# Utilisation de pd.ExcelWriter pour écrire sur plusieurs feuilles dans un seul fichier
with pd.ExcelWriter('mini_projet.xlsx') as writer:
    # Écriture de la feuille 'gounghin'
    df_gounghin.to_excel(writer, sheet_name='gounghin', index=False)
    
    # Écriture de la feuille 'A_to_P'
    df_a_to_p.to_excel(writer, sheet_name='A_to_P', index=False)

print("Le fichier 'mini_projet.xlsx' a été créé avec succès.")
print("Votre projet est terminé. Vous pouvez maintenant soumettre vos fichiers sur GitHub.")


Bibliothèques importées avec succès.
--------------------------------------------------
Lecture du fichier BF/BF.txt...
Fichier 'burkina_location.csv' créé avec succès.

Aperçu des 5 premières lignes du nouveau fichier :
        ID       location_name       lat     long
0  2282318              Pouéné   9.72908 -2.78660
1  2285251  Léraba Occidentale  10.28333 -5.11667
2  2287216           Kéléouoro   9.80748 -4.05023
3  2294066         White Volta   8.70194 -0.99056
4  2298457        Issana Bouga  10.91667 -1.18333
--------------------------------------------------
Fichier 'gounghin.csv' créé avec les données contenant 'gounghin'.

Données pour 'gounghin' :
             ID                  location_name       lat     long
153     2353306                       Gounghin  12.06677 -1.42134
7269    2360473                       Gounghin  12.62488 -1.36398
10260   2570204                       Gounghin  12.31436 -1.37900
10746  10342749                       Gounghin  12.06667 -0.15000
1075